## Basic

## Pandas

### Compose plots

In [1]:
import numpy as np
import pandas as pd
import hvplot.pandas  # noqa

np.random.seed(1)

idx = pd.date_range('1/1/2000', periods=1000)
df = pd.DataFrame(np.random.randn(1000, 4), index=idx, columns=list('ABCD')).cumsum()
df.head(2)

,A,B,C,D
2000-01-01,1.624345,-0.611756,-0.528172,-1.072969
2000-01-02,2.489753,-2.913295,1.216640,-1.834176


In [2]:
# %matplotlib inline
# df.plot()

In [3]:
df.hvplot()

# call the holoviews backend to replace the default matplotlib backend .plot()
# pd.options.plotting.backend = 'holoviews'
# df.plot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [4]:
plot1 = df['A'].hvplot.area(alpha=0.2, color='red', width=300)
plot2 = df['B'].hvplot.line(width=300)
print(plot2)

:Curve   [index]   (B)


In [5]:
plot1 + plot2 # lays out plots in a row

:Layout
   .Area.A  :Area   [index]   (A)
   .Curve.B :Curve   [index]   (B)

In [ ]:
plot1 * plot2 # cover two plots

:Overlay
   .Area.A  :Area   [index]   (A)
   .Curve.B :Curve   [index]   (B)

### Widgets-based exploration

In [7]:
from bokeh.sampledata.penguins import data as df_penguins
df_penguins.head(2)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE


In [8]:
df_penguins.hvplot.scatter(x='bill_length_mm', y='bill_depth_mm', groupby=['island', 'sex'], dynamic=False) # dynamic=False: static plot

:HoloMap   [island,sex]
   :Scatter   [bill_length_mm]   (bill_depth_mm)

### explorer

In [9]:
explorer = df_penguins.hvplot.explorer(x='bill_length_mm', y='bill_depth_mm', by=['species'])
explorer

BokehModel(combine_events=True, render_bundle={'docs_json': {'6adbff6b-0f34-420b-9230-3546bb724fef': {'version…

## Xarray

### Ploting 

In [10]:
import hvplot.xarray  # noqa
import xarray as xr

air_ds = xr.tutorial.open_dataset('air_temperature').load()
air_ds

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [11]:
air_ds['air'].isel(time=slice(0, 1000, 60)).hvplot.image(dynamic=False)

:HoloMap   [time]
   :Image   [lon,lat]   (4xDaily Air temperature at sigma level 995)

### Geographic plots 

In [12]:
import cartopy.crs as crs

proj = crs.Orthographic(-90, 30)

In [13]:
# quadmesh: quadrilateral grid mesh (global_extent=True : plot the whole world; project=True : project the data to the map)
air_ds.air.sel(time='2013-06-01 12:00').hvplot.quadmesh(
    'lon', 'lat',  # x and y coordinates (cannot change)
    projection=proj,  # projection
    project=True,  # project the data to the map
    global_extent=True,  # plot the whole world
    cmap='viridis',  # color map
    coastline=True  # plot the coastline
)

:Overlay
   .QuadMesh.I  :QuadMesh   [lon,lat]   (4xDaily Air temperature at sigma level 995)
   .Coastline.I :Feature   [Longitude,Latitude]

In [14]:
# hvplot.extension('matplotlib') # default is bokeh, directly use matplotlib for plot engine

# air_ds.air.isel(time=slice(0, 9, 3)).hvplot.quadmesh(
#     'lon', 'lat', projection=proj, project=True, global_extent=True, 
#     cmap='viridis', rasterize=True, dynamic=False, coastline=True,
#     xaxis=None, yaxis=None, width=370
# )

### Interactive Static

#### Original

In [15]:
air = air_ds.air 

# Calculate the rolling mean of the air temperature

# fix the one lat degree
LATITUDE = 30.
ROLLING_WINDOW = '1D' # rolling window size '1d

# calculate the baseline temperature
baseline = air.sel(lat=LATITUDE).mean().item()


# pipeline
pipeline = (
    air
    .sel(lat=LATITUDE)
    .to_dataframe()
    .drop(columns='lat') # drop the latitude column
    .groupby('time').mean() # group by time and calculate the mean
    .rolling(ROLLING_WINDOW).mean() # rolling window size '1d'
    - baseline # subtract the baseline temperature
)
pipeline.describe()

,air
count,2920.000000
mean,0.000418
std,3.185757
min,-6.247181
25%,-2.930566
50%,-0.344752
75%,3.222418
max,5.372583


#### hvplot & panel

In [16]:
import panel as pn

w_latitude = pn.widgets.DiscreteSlider(name='Latitude', options=list(air.lat.values))
w_rolling_window = pn.widgets.RadioButtonGroup(name='Rolling window', options=['1D', '7D', '30D'])

In [17]:
airi = air.interactive()

In [18]:
baseline = airi.sel(lat=w_latitude).mean().item()
pipeline = (
    airi
    .sel(lat=w_latitude)
    .to_dataframe()
    .drop(columns='lat')
    .groupby('time').mean()
    .rolling(w_rolling_window).mean()
    - baseline
)
pipeline.describe()

BokehModel(combine_events=True, render_bundle={'docs_json': {'2c493823-1adb-4044-823e-842a5442a7e0': {'version…

In [19]:
import hvplot.pandas  # noqa

pipeline.hvplot()

BokehModel(combine_events=True, render_bundle={'docs_json': {'2928fde2-5bd9-42dc-8a9e-abcdacfe3bf4': {'version…

### Contourf

In [20]:
air_ds

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [21]:
air_ds.hvplot.contourf()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a6ae5821-0a79-4fef-8077-3c50fd65ac14': {'version…

In [22]:
air_ds.hvplot.contourf(levels=20, geo=True, coastline=True, widget_location='left_top') # level is fix the number of scale 

BokehModel(combine_events=True, render_bundle={'docs_json': {'1427bffc-cd9e-4ae2-842e-991cbadc9cc4': {'version…

In [23]:
air_ds.mean(dim='time').hvplot.contourf(z='air', x='lon', y='lat', levels=20,
                                    clabel='T [K]', label='Mean Air temperature [K]',
                                    cmap='fire')

:Polygons   [lon,lat]   (air)

### Hist and Kde

In [24]:
# fix the lon and lat
LATITUDE = 40.
LONTITUDE = 285.
air1d = air_ds.air.sel(lon = LONTITUDE, lat = LATITUDE)
air1d.hvplot.hist()

:NdOverlay   [Variable]
   :Histogram   [air]   (Count)

In [25]:
# plot the cumulative histogram
air1d.hvplot.hist(bins=20, cumulative=True, normed=True)

:NdOverlay   [Variable]
   :Histogram   [air]   (Frequency)

In [26]:
air1d.hvplot.hist(title=f"Air Temperature over time at lat={LATITUDE}, lon={LONTITUDE}", color='gray')

:NdOverlay   [Variable]
   :Histogram   [air]   (Count)

In [27]:
air1d = air.sel(lat=[25, 50, 75])
air1d.hvplot.kde('air', by='lat', alpha=0.5)

:NdOverlay   [lat]
   :Distribution   [air]   (Density)

### Boxplot or Violin plot

In [28]:
air = air_ds.air
air.hvplot.violin(y='air', by='lat', color='lat', cmap='Category20', title="Air Temperature vs. latitude") # by is the group by the latitude (all the lon will be grouped by the latitude)

:Violin   [lat]   (4xDaily Air temperature at sigma level 995)

### Vectorfield

In [29]:
import numpy as np
import xarray as xr
import holoviews as hv
import cartopy.crs as ccrs

In [30]:
def sample_data(shape=(20, 30)):
    """
    Return ``(x, y, u, v, crs)`` of some vector data
    computed mathematically. The returned crs will be a rotated
    pole CRS, meaning that the vectors will be unevenly spaced in
    regular PlateCarree space.

    """
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)

    x = np.linspace(311.9, 391.1, shape[1])
    y = np.linspace(-23.6, 24.8, shape[0])

    x2d, y2d = np.meshgrid(x, y)
    u = 10 * (2 * np.cos(2 * np.deg2rad(x2d) + 3 * np.deg2rad(y2d + 30)) ** 2)
    v = 20 * np.cos(6 * np.deg2rad(x2d))

    return x, y, u, v, crs

In [31]:
xs, ys, U, V, crs = sample_data()

mag = np.sqrt(U**2 + V**2)
angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)

ds = xr.Dataset({'mag': xr.DataArray(mag, dims=('y', 'x'), coords={'y': ys, 'x': xs}),
                 'angle': xr.DataArray(angle, dims=('y', 'x'), coords={'y': ys, 'x': xs})}, 
                attrs={'crs': crs})
ds

<xarray.Dataset> Size: 10kB
Dimensions:  (y: 20, x: 30)
Coordinates:
  * y        (y) float64 160B -23.6 -21.05 -18.51 -15.96 ... 19.71 22.25 24.8
  * x        (x) float64 240B 311.9 314.6 317.4 320.1 ... 385.6 388.4 391.1
Data variables:
    mag      (y, x) float64 5kB 6.459 2.149 5.899 11.25 ... 22.28 22.74 22.0
    angle    (y, x) float64 5kB 1.413 0.3677 -0.9793 ... -0.9368 -1.049 -1.127
Attributes:
    crs:      +proj=ob_tran +ellps=WGS84 +a=6378137.0 +o_proj=latlon +o_lon_p...

In [32]:
ds.hvplot.vectorfield(x='x', y='y', angle='angle', mag='mag', hover=False).opts(magnitude='mag')

:VectorField   [x,y]   (angle,mag)

In [33]:
ds.hvplot.vectorfield(x='x', y='y', angle='angle', mag='mag',
                      hover=False, geo=True, tiles="CartoLight")

:Overlay
   .WMTS.I        :WMTS   [Longitude,Latitude]
   .VectorField.I :VectorField   [x,y]   (angle,mag)